In [18]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import os
import sys
from google.colab import drive

In [19]:
#Conectar al drive local
drive.mount('/content/drive')

#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_helath'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

%run funciones.ipynb

Mounted at /content/drive


In [24]:
# Etiquetas multiclase en el orden correcto
class_labels = ['glioma', 'meningioma', 'pituitary', 'notumor']

if __name__ == "__main__":

    # Cargar imágenes de despliegue
    path = 'data/salidas/despliegue'
    x, _, files = cargar_imagenes(path)

    # Convertir a array
    x = np.array(x)

    # Redimensionar imágenes a (99, 75)
    new_size = (99, 75)
    x = np.array([cv2.resize(img, new_size, interpolation=cv2.INTER_AREA) for img in x])

    # Escalar a [0,1]
    x = x.astype('float32') / 255.0

    # Eliminar extensión de archivos
    pacientes = [name.rsplit('.', 1)[0] for name in files]

    # Cargar el modelo CNN multiclase entrenado
    modelo = tf.keras.models.load_model('data/salidas/best_model.keras')

    # Predecir probabilidades
    prob = modelo.predict(x)

    # Predecir la clase (índice de clase con mayor probabilidad)
    y_pred = np.argmax(prob, axis=1)

    # Convertir índice a nombre de clase
    clases = [class_labels[i] for i in y_pred]

    # Guardar resultados en DataFrame
    resultados = pd.DataFrame({
        "paciente": pacientes,
        "clasificacion": clases
    })

    # Exportar a Excel
    resultados.to_excel('data/salidas/clasificaciones_multiclase.xlsx', index=False)
    print("Resultados exportados a 'clasificaciones_multiclase.xlsx'")

ValueError: File not found: filepath=data/salidas/best_model.keras. Please ensure the file is an accessible `.keras` zip file.